In [1]:
%matplotlib tk
import cv2.cv2 as cv2
import matplotlib.pyplot as plt
import numpy as np

from noise import add_noise, NoiseTypes

# Ejericio 1.1: Filtrado anisotrópico un solo canal

Se he implementado en python la función matlab que se incluye como parte de los ejercicios.

In [2]:
def dif_aniso(im: np.ndarray, niter: int, k: float, l: float, option: int) -> np.ndarray:
    assert im.ndim == 2, "Image must be gray (2 channel)"

    rows, cols = im.shape
    imdif = im.copy()

    for i in range(niter):
        # pads image with zeros
        imdifm = np.pad(imdif, (1,), constant_values=0, mode='constant')

        # gradients in 4 directions
        deltaN = imdifm[0:rows, 1:cols + 1] - imdif
        deltaS = imdifm[2:rows + 2, 1:cols + 1] - imdif
        deltaE = imdifm[1:rows + 1, 2:cols + 2] - imdif
        deltaW = imdifm[1:rows + 1, 0:cols] - imdif

        # conductance
        if option == 1:
            cN = np.exp(-(deltaN / k) ** 2)
            cS = np.exp(-(deltaS / k) ** 2)
            cE = np.exp(-(deltaE / k) ** 2)
            cW = np.exp(-(deltaW / k) ** 2)
        elif option == 2:
            cN = 1. / np.exp(1. + (deltaN / k) ** 2)
            cS = 1. / np.exp(1. + (deltaS / k) ** 2)
            cE = 1. / np.exp(1. + (deltaE / k) ** 2)
            cW = 1. / np.exp(1. + (deltaW / k) ** 2)

        imdif = imdif + (l * (cN * deltaN + cS * deltaS + cE * deltaE + cW * deltaW))
        pass

    return imdif

In [3]:
def read_image(file):
    image = cv2.imread(file, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float64)
    return image

## Determinando el parámetro K

Para determinar el valor del parámetro K, se han generado las imágenes de gradientes en las 4 direcciones y se ha analizado el valor del gradiente en las zonas de bajo contraste cuyos bordes queremos mantener.

De esta forma hemos determinado que el valor que búscamos para el parámetro K es 8.

In [4]:
def show_gradients(image_file):
    image = read_image(image_file)
    image = add_noise(image=image, noise_type=NoiseTypes.RICIAN_NOISE, intensity=0.1)
    rows, cols = image.shape

    # pads image with zeros
    imdifm = np.pad(image, (1,), constant_values=0, mode='constant')

    # gradients in 4 directions
    deltaN = imdifm[0:rows, 1:cols + 1] - image
    deltaS = imdifm[2:rows + 2, 1:cols + 1] - image
    deltaE = imdifm[1:rows + 1, 2:cols + 2] - image
    deltaW = imdifm[1:rows + 1, 0:cols] - image

    fig, ax = plt.subplots(2, 2, figsize=(14.4, 14.4), tight_layout={
        'rect': [0, 0, 1, 0.98] }
    )
    ax[0,0].imshow(deltaN, cmap='gray')
    ax[0,1].imshow(deltaS, cmap='gray')
    ax[1,0].imshow(deltaE, cmap='gray')
    ax[1,1].imshow(deltaW, cmap='gray')
    fig.show()

show_gradients("materiales/T1.png")
show_gradients("materiales/T2.png")

## Realizando el filtrado en T1.png

Se ha realizado y presentado el filtrado en T1.png con el parámetro K deducido con el método anteriormente comentado y numero de iteraciones y valor de lambda probados de forma empírica.

In [12]:
def filter_image(image_file):
    imageT1 = read_image(image_file)
    noise_image = add_noise(image=imageT1, noise_type=NoiseTypes.RICIAN_NOISE, intensity=0.1)
    denoised_image = dif_aniso(im=noise_image, niter=10, k=8, l=0.40, option=1)

    fig, ax = plt.subplots(1, 3, figsize=(8.4, 14.4), tight_layout={
            'rect': [0, 0, 1, 0.98]
        }
    )
    ax[0].imshow(imageT1, cmap='gray')
    ax[0].set_title('original T1')
    ax[1].imshow(noise_image, cmap='gray')
    ax[1].set_title('ruidosa T1')
    ax[2].imshow(denoised_image, cmap='gray')
    ax[2].set_title("denoised T1")
    fig.show()

filter_image("materiales/T1.png")

Y hacemos lo mismo para T2

In [13]:
filter_image("materiales/T2.png")

# Ejercicio 1.2: Filtrado anisotrópico multicanal

Se ha modificado la función dif_aniso (ahora dif_aniso_mc) para aceptar dos imágenes, y realizar el cálculo del coeficiente empleando la fórmula propuesta por Gerig, Kikins, Kübler y Jolesz:

$$ c(\vec{x}, t) = \sqrt{\nabla I_1^2 + \nabla I_2^2} $$

In [14]:
def dif_aniso_mc(im1: np.ndarray, im2: np.ndarray, niter: int, k: float, l: float, option: int) -> np.ndarray:
    assert im1.ndim == 2, "Image must be gray (2 channel)"
    assert im2.ndim == 2, "Image 2 must be gray (2 channel)"
    assert im1.shape == im2.shape, "Both images must be same size"

    rows, cols = im1.shape
    imdif1 = im1.copy()
    imdif2 = im2.copy()

    for i in range(niter):
        # pads images 1 and 2 with zeros
        imdifm1 = np.pad(imdif1, (1,), constant_values=0, mode='constant')
        imdifm2 = np.pad(imdif2, (1,), constant_values=0, mode='constant')

        # gradients in 4 directions image 1
        deltaN1 = imdifm1[0:rows, 1:cols + 1] - imdif1
        deltaS1 = imdifm1[2:rows + 2, 1:cols + 1] - imdif1
        deltaE1 = imdifm1[1:rows + 1, 2:cols + 2] - imdif1
        deltaW1 = imdifm1[1:rows + 1, 0:cols] - imdif1

        # gradients in 4 directions image 2
        deltaN2 = imdifm2[0:rows, 1:cols + 1] - imdif2
        deltaS2 = imdifm2[2:rows + 2, 1:cols + 1] - imdif2
        deltaE2 = imdifm2[1:rows + 1, 2:cols + 2] - imdif2
        deltaW2 = imdifm2[1:rows + 1, 0:cols] - imdif2

        # combine gradientes
        deltaN = np.sqrt(deltaN1 ** 2 + deltaN2 ** 2)
        deltaS = np.sqrt(deltaS1 ** 2 + deltaS2 ** 2)
        deltaE = np.sqrt(deltaE1 ** 2 + deltaE2 ** 2)
        deltaW = np.sqrt(deltaW1 ** 2 + deltaW2 ** 2)

        # conductance
        if option == 1:
            cN = np.exp(-(deltaN / k) ** 2)
            cS = np.exp(-(deltaS / k) ** 2)
            cE = np.exp(-(deltaE / k) ** 2)
            cW = np.exp(-(deltaW / k) ** 2)
        elif option == 2:
            cN = 1. / np.exp(1. + (deltaN / k) ** 2)
            cS = 1. / np.exp(1. + (deltaS / k) ** 2)
            cE = 1. / np.exp(1. + (deltaE / k) ** 2)
            cW = 1. / np.exp(1. + (deltaW / k) ** 2)

        imdif1 = imdif1 + (l * (cN * deltaN + cS * deltaS + cE * deltaE + cW * deltaW))
        imdif2 = imdif2 + (l * (cN * deltaN + cS * deltaS + cE * deltaE + cW * deltaW))

    return imdif1, imdif2

Y generado la salida empleando ambas imágenes

In [21]:
def filter_images(image_file1, image_file2):
    image1 = read_image(image_file1)
    image2 = read_image(image_file2)
    noise_image1 = add_noise(image=image1, noise_type=NoiseTypes.RICIAN_NOISE, intensity=0.1)
    noise_image2 = add_noise(image=image2, noise_type=NoiseTypes.RICIAN_NOISE, intensity=0.1)
    denoised_image1, denoised_image2 = dif_aniso_mc(im1=noise_image1, im2=noise_image2, niter=10, k=8, l=0.25, option=1)

    fig, ax = plt.subplots(2, 3, figsize=(8.4, 14.4), tight_layout={
            'rect': [0, 0, 1, 0.98]
        }
    )
    ax[0,0].imshow(image1, cmap='gray')
    ax[0,0].set_title('original T1')
    ax[0,1].imshow(noise_image1, cmap='gray')
    ax[0,1].set_title('ruidosa T1')
    ax[0,2].imshow(denoised_image1, cmap='gray')
    ax[0,2].set_title("denoised T1")

    ax[1,0].imshow(image2, cmap='gray')
    ax[1,0].set_title('original T2')
    ax[1,1].imshow(noise_image2, cmap='gray')
    ax[1,1].set_title('ruidosa T2')
    ax[1,2].imshow(denoised_image2, cmap='gray')
    ax[1,2].set_title("denoised T2")
    fig.show()

filter_images("materiales/T1.png", "materiales/T2.png")